In [7]:
#!/usr/bin/env python3
# subscriber.py
import cv2, zmq, numpy as np, time, threading, queue, traceback, os, sys, cvzone
from cvzone.ColorModule import ColorFinder
from collections import deque, defaultdict

# ---------- Config ----------
ZMQ_ADDR = "tcp://localhost:5555"
SUB_TOPICS = [b"kreo1", b"kreo2"]
FPS_WINDOW = 1.0        # seconds for fps moving window
DISPLAY_FPS = 20
VISUALIZE = True     # show tiled view window
MIN_BLOB_AREA = 80        # tiny specks ignored
MAX_BLOB_AREA = 20000     # huge regions ignored

orange_hsvVals = {'hmin': 0, 'smin': 94, 'vmin': 156, 'hmax': 12, 'smax': 255, 'vmax': 255}
purple_hsvVals = {'hmin': 113, 'smin': 78, 'vmin': 3, 'hmax': 129, 'smax': 255, 'vmax': 255}

def mask_orange(hsv, lab):
    h, s, v = cv2.split(hsv)
    # Hue wraps in OpenCV: orange roughly 5-22 (0..179)
    lower_h = 5; upper_h = 22
    m_h = cv2.inRange(h, lower_h, upper_h)
    m_s = cv2.inRange(s, 110, 255)   # require decent saturation
    m_v = cv2.inRange(v, 90, 255)    # reasonably bright
    return cv2.bitwise_and(cv2.bitwise_and(m_h, m_s), m_v)

def mask_purple(hsv, lab):
    # Purple/blue: hue around 120-150 in many lights but mixed lighting causes shifts.
    # Use Lab a,b clues too: purple tends to show particular (a,b) signature.
    h, s, v = cv2.split(hsv)
    l, a, b = cv2.split(lab)
    m_h = cv2.inRange(h, 120, 160)            # wide hue band
    m_s = cv2.inRange(s, 40, 255)             # allow lower sat since purple can be dark
    m_v = cv2.inRange(v, 30, 220)             # dark to mid brightness
    # Lab b-channel for purple tends to be low/negative in some lighting; accept wide band
    m_b = cv2.inRange(b, 0, 160)              # permissive
    mask = cv2.bitwise_and(m_h, m_s)
    mask = cv2.bitwise_and(mask, m_v)
    mask = cv2.bitwise_or(mask, m_b)   # include lab cue
    return mask

def mask_brown(hsv, lab):
    # Brown: low brightness, low-medium saturation, hue around 5-30 or 10-30 depending on wood floor
    h, s, v = cv2.split(hsv)
    l, a, b = cv2.split(lab)
    m_h = cv2.inRange(h, 5, 30)
    m_s = cv2.inRange(s, 30, 200)
    m_v = cv2.inRange(v, 20, 140)  # dark
    # Lab 'a' tends to be positive for brown; use permissive range
    m_a = cv2.inRange(a, 120, 200)
    mask = cv2.bitwise_and(m_h, m_s)
    mask = cv2.bitwise_and(mask, m_v)
    mask = cv2.bitwise_or(mask, m_a)
    return mask


K_OPEN = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
K_CLOSE = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7))

myColorFinder = ColorFinder(trackBar=True)

class BallDetectorThread(threading.Thread):
    def __init__(self, cam_name, frame_queue, result_dict, lock):
        super().__init__(daemon=True)
        self.cam_name = cam_name
        self.frame_queue = frame_queue
        self.result = result_dict
        self.lock = lock
        self.stop_flag = False

    def run(self):
        print(f"[{self.cam_name}] Detector thread started.")
        while not self.stop_flag:
            try:
                frame,ts = self.frame_queue.get(timeout=0.1)
            except queue.Empty:
                continue
            try:
                img = frame.copy()

                detection = None
                with self.lock:
                    self.result[self.cam_name] = detection
            except Exception as e:
                print(f"[ERROR-{self.cam_name}] detector exception:", e)
                traceback.print_exc()

        print(f"[DETECT-{self.cam_name}] Detector thread stopped")

    def stop(self):
        self.stop_flag = True

# ---------- Helpers ----------
def fmt_ts(ts):
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(ts)) + f".{int((ts%1)*1000):03d}"

def recv_latest(sub):
    msg = None
    while True:
        try:
            msg = sub.recv_multipart(flags=zmq.NOBLOCK)
        except zmq.Again:
            break
    return msg

def update_fps(camera, cam_ts):
    dq = fps_windows[camera]
    dq.append(cam_ts)
    # pop older than window
    while dq and (cam_ts - dq[0]) > FPS_WINDOW:
        dq.popleft()
    fps = len(dq) / FPS_WINDOW
    return fps

def circularity(contour):
    area = cv2.contourArea(contour)
    perim = cv2.arcLength(contour, True)
    if perim <= 0: return 0.0
    return 4.0 * np.pi * area / (perim*perim)


# ---------- ZMQ subscriber ----------
ctx = zmq.Context()
sub = ctx.socket(zmq.SUB)
sub.connect(ZMQ_ADDR)
sub.setsockopt(zmq.RCVHWM, 1)
sub.setsockopt(zmq.CONFLATE, 1)  # keep only last message
sub.setsockopt(zmq.LINGER, 0)


# ---- ACTIVE FLUSH ----
flushed = 0
while True:
    try:
        sub.recv_multipart(flags=zmq.NOBLOCK)
        flushed += 1
    except zmq.Again:
        break
if flushed > 0:
    print(f"[Subscriber] Flushed {flushed} stale messages.")
for t in SUB_TOPICS:
    sub.setsockopt(zmq.SUBSCRIBE, t)

# per-camera state
frames = {}
fps_windows = defaultdict(lambda: deque())   # deque of capture times
frame_queues = {t.decode(): queue.Queue(maxsize=1) for t in SUB_TOPICS}
detect_results = {}
detect_lock = threading.Lock()
det_threads = {}

for t in SUB_TOPICS:
    cam_name = t.decode()
    dt = BallDetectorThread(cam_name, frame_queues[cam_name], detect_results, detect_lock)
    dt.start()
    det_threads[cam_name] = dt


print("[Subscriber] connected, waiting for frames... (Press ESC to exit)")

last_show = time.time()
# ---------- Main loop ----------
try:
    while True:
        parts = recv_latest(sub)
        if parts is None:
            if VISUALIZE:
                cv2.waitKey(1)
            continue

        # unpack message
        topic = parts[0]
        cam = topic.decode()

        if len(parts) >= 3:
            ts_part = parts[1]
            jpg_part = parts[2]
        else:
            ts_part = None
            jpg_part = parts[1]

        recv_time = time.time()

        try:
            cam_ts = float(ts_part.decode()) if ts_part else recv_time
        except:
            cam_ts = recv_time

        img = cv2.imdecode(np.frombuffer(jpg_part, np.uint8), cv2.IMREAD_COLOR)
        if img is None:
            continue

        fps = update_fps(cam, cam_ts)

        frames[cam] = {
            "img": img,
            "cam_ts": cam_ts,
            "fps": fps,
        }

        fq = frame_queues[cam]
        try:
            fq.get_nowait()  # clear old
        except queue.Empty:
            pass
        try:
            fq.put_nowait((img.copy(), cam_ts))
        except queue.Full:
            pass

        # If we have at least 2 cameras, compute drift and show tiled view
        if all(k in frames for k in [t.decode() for t in SUB_TOPICS]):
            cams = [t.decode() for t in SUB_TOPICS]
            left = frames[cams[0]]
            right = frames[cams[1]]

            # compute drift in ms between corrected timestamps
            drift_ms = abs(left["cam_ts"] - right["cam_ts"]) * 1000.0

            # overlay text on each image
            def overlay(info, cam_name):
                im = info["img"].copy()
                y = 20
                cv2.putText(im, f"{cam_name}", (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,32,20), 2)
                cv2.putText(im, f"FPS: {info['fps']:.1f}", (10, y+26), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (14,117,5), 2)
                cv2.putText(im, f"cam_ts: {fmt_ts(info['cam_ts'])}", (10, y+52), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (5,12,117), 1)

                # draw detection if exists
                _, mask1 = myColorFinder.update(im, orange_hsvVals)
                

                return _

            if VISUALIZE and time.time()-last_show > 1.0/DISPLAY_FPS:
                left_im = overlay(left, cams[0])
                right_im = overlay(right, cams[1])

                # resize to same height and tile horizontally
                h = max(left_im.shape[0], right_im.shape[0])
                right_resized = cv2.resize(right_im, (left_im.shape[1], h))
                tile = np.hstack([left_im, right_resized])

                # Draw drift and timestamp summary on top-left of tiled image
                cv2.putText(tile, f"Drift: {drift_ms:.1f} ms", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
                cv2.putText(tile, f"Host now: {fmt_ts(time.time())}", (10, 44), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

                last_show = time.time()
                cv2.imshow("Both Cameras (tiled)", tile)
            elif not VISUALIZE:
                with detect_lock:
                    s0 = detect_results.get(cams[0], None)
                    s1 = detect_results.get(cams[1], None)
                    t0 = s0["label"] if s0 else "none"
                    t1 = s1["label"] if s1 else "none"
                status = (f"Drift {drift_ms:.1f} ms | {cams[0]} FPS:{left['fps']:.1f} | "
                          f"{cams[1]} FPS:{right['fps']:.1f} | Labels: {cams[0]}:{t0} {cams[1]}:{t1}")
                sys.stdout.write("\r" + status + " " * 20)
                sys.stdout.flush()
            

        if cv2.waitKey(1) & 0xFF == 27:
            break

except KeyboardInterrupt:
    pass
finally:
    cv2.destroyAllWindows()
    sub.close()
    ctx.term()
    print("\n[Subscriber] exited cleanly.")


[kreo1] Detector thread started.
[kreo2] Detector thread started.
[Subscriber] connected, waiting for frames... (Press ESC to exit)
{'hmin': 0, 'smin': 0, 'vmin': 0, 'hmax': 179, 'smax': 255, 'vmax': 255}
{'hmin': 0, 'smin': 0, 'vmin': 0, 'hmax': 179, 'smax': 255, 'vmax': 255}
{'hmin': 0, 'smin': 0, 'vmin': 0, 'hmax': 179, 'smax': 255, 'vmax': 255}
{'hmin': 0, 'smin': 0, 'vmin': 0, 'hmax': 179, 'smax': 255, 'vmax': 255}
{'hmin': 0, 'smin': 0, 'vmin': 0, 'hmax': 179, 'smax': 255, 'vmax': 255}
{'hmin': 0, 'smin': 0, 'vmin': 0, 'hmax': 179, 'smax': 255, 'vmax': 255}
{'hmin': 0, 'smin': 0, 'vmin': 0, 'hmax': 179, 'smax': 255, 'vmax': 255}
{'hmin': 0, 'smin': 0, 'vmin': 0, 'hmax': 179, 'smax': 255, 'vmax': 255}
{'hmin': 0, 'smin': 0, 'vmin': 0, 'hmax': 179, 'smax': 255, 'vmax': 255}
{'hmin': 0, 'smin': 0, 'vmin': 0, 'hmax': 179, 'smax': 255, 'vmax': 255}
{'hmin': 0, 'smin': 0, 'vmin': 0, 'hmax': 179, 'smax': 255, 'vmax': 255}
{'hmin': 0, 'smin': 0, 'vmin': 0, 'hmax': 179, 'smax': 255, 'vmax

In [9]:
#!/usr/bin/env python3
# subscriber.py
import cv2, zmq, numpy as np, time, threading, queue, traceback, os, sys, cvzone
from cvzone.ColorModule import ColorFinder
from collections import deque, defaultdict

# ---------- Config ----------
ZMQ_ADDR = "tcp://localhost:5555"
SUB_TOPICS = [b"kreo1", b"kreo2"]
FPS_WINDOW = 1.0        # seconds for fps moving window
DISPLAY_FPS = 20
VISUALIZE = True     # show tiled view window

orange_hsvVals = {'hmin': 149, 'smin': 69, 'vmin': 82, 'hmax': 177, 'smax': 229, 'vmax': 252}
purple_hsvVals = {'hmin': 113, 'smin': 78, 'vmin': 3, 'hmax': 129, 'smax': 255, 'vmax': 255}

K_OPEN = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
K_CLOSE = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7))

myColorFinder = ColorFinder(trackBar=False)

class BallDetectorThread(threading.Thread):
    def __init__(self, cam_name, frame_queue, result_dict, lock):
        super().__init__(daemon=True)
        self.cam_name = cam_name
        self.frame_queue = frame_queue
        self.result = result_dict
        self.lock = lock
        self.stop_flag = False

    def run(self):
        print(f"[{self.cam_name}] Detector thread started.")
        while not self.stop_flag:
            try:
                frame,ts = self.frame_queue.get(timeout=0.1)
            except queue.Empty:
                continue
            try:
                img = frame.copy()

                detection = None
                with self.lock:
                    self.result[self.cam_name] = detection
            except Exception as e:
                print(f"[ERROR-{self.cam_name}] detector exception:", e)
                traceback.print_exc()

        print(f"[DETECT-{self.cam_name}] Detector thread stopped")

    def stop(self):
        self.stop_flag = True

# ---------- Helpers ----------
def fmt_ts(ts):
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(ts)) + f".{int((ts%1)*1000):03d}"

def recv_latest(sub):
    msg = None
    while True:
        try:
            msg = sub.recv_multipart(flags=zmq.NOBLOCK)
        except zmq.Again:
            break
    return msg

def update_fps(camera, cam_ts):
    dq = fps_windows[camera]
    dq.append(cam_ts)
    # pop older than window
    while dq and (cam_ts - dq[0]) > FPS_WINDOW:
        dq.popleft()
    fps = len(dq) / FPS_WINDOW
    return fps


# ---------- ZMQ subscriber ----------
ctx = zmq.Context()
sub = ctx.socket(zmq.SUB)
sub.connect(ZMQ_ADDR)
sub.setsockopt(zmq.RCVHWM, 1)
sub.setsockopt(zmq.CONFLATE, 1)  # keep only last message
sub.setsockopt(zmq.LINGER, 0)


# ---- ACTIVE FLUSH ----
flushed = 0
while True:
    try:
        sub.recv_multipart(flags=zmq.NOBLOCK)
        flushed += 1
    except zmq.Again:
        break
if flushed > 0:
    print(f"[Subscriber] Flushed {flushed} stale messages.")
for t in SUB_TOPICS:
    sub.setsockopt(zmq.SUBSCRIBE, t)

# per-camera state
frames = {}
fps_windows = defaultdict(lambda: deque())   # deque of capture times
frame_queues = {t.decode(): queue.Queue(maxsize=1) for t in SUB_TOPICS}
detect_results = {}
detect_lock = threading.Lock()
det_threads = {}

for t in SUB_TOPICS:
    cam_name = t.decode()
    dt = BallDetectorThread(cam_name, frame_queues[cam_name], detect_results, detect_lock)
    dt.start()
    det_threads[cam_name] = dt


print("[Subscriber] connected, waiting for frames... (Press ESC to exit)")

last_show = time.time()
# ---------- Main loop ----------
try:
    while True:
        parts = recv_latest(sub)
        if parts is None:
            if VISUALIZE:
                cv2.waitKey(1)
            continue

        # unpack message
        topic = parts[0]
        cam = topic.decode()

        if len(parts) >= 3:
            ts_part = parts[1]
            jpg_part = parts[2]
        else:
            ts_part = None
            jpg_part = parts[1]

        recv_time = time.time()

        try:
            cam_ts = float(ts_part.decode()) if ts_part else recv_time
        except:
            cam_ts = recv_time

        img = cv2.imdecode(np.frombuffer(jpg_part, np.uint8), cv2.IMREAD_COLOR)
        if img is None:
            continue

        fps = update_fps(cam, cam_ts)

        frames[cam] = {
            "img": img,
            "cam_ts": cam_ts,
            "fps": fps,
        }

        fq = frame_queues[cam]
        try:
            fq.get_nowait()  # clear old
        except queue.Empty:
            pass
        try:
            fq.put_nowait((img.copy(), cam_ts))
        except queue.Full:
            pass

        # If we have at least 2 cameras, compute drift and show tiled view
        if all(k in frames for k in [t.decode() for t in SUB_TOPICS]):
            cams = [t.decode() for t in SUB_TOPICS]
            left = frames[cams[0]]
            right = frames[cams[1]]

            # compute drift in ms between corrected timestamps
            drift_ms = abs(left["cam_ts"] - right["cam_ts"]) * 1000.0

            # overlay text on each image
            def overlay(info, cam_name):
                im = info["img"].copy()
                y = 20
                cv2.putText(im, f"{cam_name}", (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,32,20), 2)
                cv2.putText(im, f"FPS: {info['fps']:.1f}", (10, y+26), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (14,117,5), 2)
                cv2.putText(im, f"cam_ts: {fmt_ts(info['cam_ts'])}", (10, y+52), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (5,12,117), 1)

                # draw detection if exists
                _, mask1 = myColorFinder.update(im, orange_hsvVals)
                _, mask2 = myColorFinder.update(im, purple_hsvVals)
                combined_mask = cv2.bitwise_or(mask1,mask2)
                
                # imgContours, contours = cvzone.findContours(img,combined_mask,minArea=100)
                contours, heirarchy = cv2.findContours(combined_mask,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
                min_area_threshold = 100
                filtered_contours = []

                for contour in contours:
                    area = cv2.contourArea(contour)
                    if area >= min_area_threshold:
                        filtered_contours.append(contour)

                # --- End of filtering logic ---

                print(f"Total Contours found: {len(contours)}")
                print(f"Contours remaining after filtering (min area >= 75): {len(filtered_contours)}")

                imgContours = im.copy() 

                for contour in filtered_contours:
                    x, y, w, h = cv2.boundingRect(contour)
                    cv2.rectangle(imgContours, (x, y), (x + w, y + h), (0, 255, 0), 3) 

                    M = cv2.moments(contour)
                    if M["m00"] != 0: 
                        cX = int(M["m10"] / M["m00"])
                        cY = int(M["m01"] / M["m00"])
                        cv2.circle(imgContours, (cX, cY), 5, (0, 0, 255), -1) 
                # imgContours = cv2.resize(imgContours,(0,0),None,0.7,0.7)

                return imgContours

            if VISUALIZE and time.time()-last_show > 1.0/DISPLAY_FPS:
                left_im = overlay(left, cams[0])
                right_im = overlay(right, cams[1])

                # resize to same height and tile horizontally
                h = max(left_im.shape[0], right_im.shape[0])
                right_resized = cv2.resize(right_im, (left_im.shape[1], h))
                tile = np.hstack([left_im, right_resized])

                # Draw drift and timestamp summary on top-left of tiled image
                cv2.putText(tile, f"Drift: {drift_ms:.1f} ms", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
                cv2.putText(tile, f"Host now: {fmt_ts(time.time())}", (10, 44), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

                last_show = time.time()
                cv2.imshow("Both Cameras (tiled)", tile)
            elif not VISUALIZE:
                with detect_lock:
                    s0 = detect_results.get(cams[0], None)
                    s1 = detect_results.get(cams[1], None)
                    t0 = s0["label"] if s0 else "none"
                    t1 = s1["label"] if s1 else "none"
                status = (f"Drift {drift_ms:.1f} ms | {cams[0]} FPS:{left['fps']:.1f} | "
                          f"{cams[1]} FPS:{right['fps']:.1f} | Labels: {cams[0]}:{t0} {cams[1]}:{t1}")
                sys.stdout.write("\r" + status + " " * 20)
                sys.stdout.flush()
            

        if cv2.waitKey(1) & 0xFF == 27:
            break

except KeyboardInterrupt:
    pass
finally:
    cv2.destroyAllWindows()
    sub.close()
    ctx.term()
    print("\n[Subscriber] exited cleanly.")


[kreo1] Detector thread started.
[kreo2] Detector thread started.
[Subscriber] connected, waiting for frames... (Press ESC to exit)
Total Contours found: 613
Contours remaining after filtering (min area >= 75): 1
Total Contours found: 76
Contours remaining after filtering (min area >= 75): 1
Total Contours found: 520
Contours remaining after filtering (min area >= 75): 2
Total Contours found: 80
Contours remaining after filtering (min area >= 75): 1
Total Contours found: 607
Contours remaining after filtering (min area >= 75): 1
Total Contours found: 87
Contours remaining after filtering (min area >= 75): 4
Total Contours found: 583
Contours remaining after filtering (min area >= 75): 1
Total Contours found: 87
Contours remaining after filtering (min area >= 75): 5
Total Contours found: 588
Contours remaining after filtering (min area >= 75): 2
Total Contours found: 72
Contours remaining after filtering (min area >= 75): 1
Total Contours found: 606
Contours remaining after filtering (m

In [2]:
import cv2
import numpy as np
import sys
import json

# Camera Config
CAM_ID = 4  # Change this to 2, 4, or proper index for your cameras to test each one
WINDOW_NAME = "Ball Tuner"

def nothing(x):
    pass

# Initialize Window with Sliders
cv2.namedWindow(WINDOW_NAME)
cv2.createTrackbar("H Min", WINDOW_NAME, 0, 179, nothing)
cv2.createTrackbar("S Min", WINDOW_NAME, 0, 255, nothing)
cv2.createTrackbar("V Min", WINDOW_NAME, 0, 255, nothing)
cv2.createTrackbar("H Max", WINDOW_NAME, 179, 179, nothing)
cv2.createTrackbar("S Max", WINDOW_NAME, 255, 255, nothing)
cv2.createTrackbar("V Max", WINDOW_NAME, 255, 255, nothing)

# Set default "Neon Orange" starting point
cv2.setTrackbarPos("H Min", WINDOW_NAME, 0)
cv2.setTrackbarPos("S Min", WINDOW_NAME, 100)
cv2.setTrackbarPos("V Min", WINDOW_NAME, 100)
cv2.setTrackbarPos("H Max", WINDOW_NAME, 25)
cv2.setTrackbarPos("S Max", WINDOW_NAME, 255)
cv2.setTrackbarPos("V Max", WINDOW_NAME, 255)

cap = cv2.VideoCapture(CAM_ID, cv2.CAP_V4L2)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cap.set(cv2.CAP_PROP_FPS, 60)

print("Adjust sliders until the BALL is white and everything else is black.")
print("Press 'q' to quit and print values.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to read camera")
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    h_min = cv2.getTrackbarPos("H Min", WINDOW_NAME)
    s_min = cv2.getTrackbarPos("S Min", WINDOW_NAME)
    v_min = cv2.getTrackbarPos("V Min", WINDOW_NAME)
    h_max = cv2.getTrackbarPos("H Max", WINDOW_NAME)
    s_max = cv2.getTrackbarPos("S Max", WINDOW_NAME)
    v_max = cv2.getTrackbarPos("V Max", WINDOW_NAME)

    lower = np.array([h_min, s_min, v_min])
    upper = np.array([h_max, s_max, v_max])

    mask = cv2.inRange(hsv, lower, upper)
    result = cv2.bitwise_and(frame, frame, mask=mask)

    # Stack for easy viewing
    mask_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    combined = np.hstack([frame, mask_bgr, result])
    
    # Resize to fit screen
    scale = 0.5
    h, w = combined.shape[:2]
    small = cv2.resize(combined, (int(w*scale), int(h*scale)))

    cv2.imshow(WINDOW_NAME, small)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("\n=== FINAL VALUES ===")
        print(f"'hmin': {h_min}, 'smin': {s_min}, 'vmin': {v_min}, 'hmax': {h_max}, 'smax': {s_max}, 'vmax': {v_max}")
        break

cap.release()
cv2.destroyAllWindows()

Adjust sliders until the BALL is white and everything else is black.
Press 'q' to quit and print values.

=== FINAL VALUES ===
'hmin': 0, 'smin': 100, 'vmin': 100, 'hmax': 25, 'smax': 255, 'vmax': 255
